#Código original

In [ ]:
# coeficientes de difusividade térmica

carbono_pirolitico = 1220 # similar ao grafite
diamante = 1100
carbono = 216.5 # 25°C
helio = 190 # 300 K, 1 atm
prata = 165.63  # puro (99.9%)
hidrogenio = 160 # 300 K, 1 atm
ouro = 127
cobre = 111 # 25°C
aluminio = 97
vapor_agua = 23.38
ferro = 23
ar = 19
tijolo = 0.52
vidro = 0.34
agua = 0.143 # 25°C
borracha = 0.1
nylon = 0.09
madeira = 0.082

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation

linhas = 50
colunas = 50
iteracoes = 602


def init():
    # criando a matriz
    u = np.empty((iteracoes, linhas, colunas)) #u[tempo][x][y]

    u_inicial = 25 # 25°C - temperatura ambiente

    # inicializando a matriz na condição inicial
    u.fill(u_inicial)

    u_baixo = 100.0
    u_esq = 50.0
    u_cima = 0.0
    u_dir = 0.0

    # inicializando as condições das bordas (o : na primeira dimensao significa que está tendo incidencia contínua nesses pontos da superficie)
    u[:, :, :1] = u_esq
    u[:, (linhas-1):, :] = u_baixo

    return u

def calcular(u, coeficiente):
    alpha = coeficiente
    delta_x = 1

    delta_t = ((delta_x-0.05) ** 2)/(4 * (alpha+0.05)) # pequeno ruído no alfa e no delta x para evitar que eles sejam cancelados na hora de calcular o gamma
    gamma = (alpha * delta_t) / (delta_x ** 2)

    for k in range(0, iteracoes-1, 1):
        for i in range(1, linhas-1, delta_x):
            for j in range(1, colunas-1, delta_x):
                u[k + 1, i, j] = gamma * (u[k][i+1][j] + u[k][i-1][j] + u[k][i][j+1] + u[k][i][j-1] - 4*(u[k][i][j])) + u[k][i][j]

    return u

def run(pontos, iteracoes):
    for k in range(0, iteracoes, 100):
        frame = pontos[k] # pegando todos os pontos em um determinado tempo k
        fig, ax = plt.subplots()
        ax.clear()
        ax.set_title(f"Temperatura na unidade de tempo k = {k}")
        ax.set_xlabel("x")
        ax.set_ylabel("y")
        c = ax.imshow(frame, interpolation='bilinear', cmap=plt.cm.jet, vmin=0, vmax=100)
        cbar = fig.colorbar(c)
        cbar.set_label("Temperatura (°C)")
        plt.show()


u = init()
# cálculo das temperaturas em todos os pontos
u = calcular(u, diamante)

#run(u, iteracoes)

#1º passo: Receber nº de iterações, largura e altura por parâmetro

In [ ]:
def init(iteracoes, linhas, colunas):
    # criando a matriz
    u = np.empty((iteracoes, linhas, colunas)) #u[tempo][x][y]

    u_inicial = 25 # 25°C - temperatura ambiente

    # inicializando a matriz na condição inicial
    u.fill(u_inicial)

    u_baixo = 100.0
    u_esq = 50.0
    u_cima = 0.0
    u_dir = 0.0

    # inicializando as condições das bordas (o : na primeira dimensao significa que está tendo incidencia contínua nesses pontos da superficie)
    u[:, :, :1] = u_esq
    u[:, (linhas-1):, :] = u_baixo

    return u

def calcular(u, coeficiente):
    iteracoes, linhas, colunas = u.shape

    alpha = coeficiente
    delta_x = 1

    delta_t = ((delta_x-0.05) ** 2)/(4 * (alpha+0.05)) # pequeno ruído no alfa e no delta x para evitar que eles sejam cancelados na hora de calcular o gamma
    gamma = (alpha * delta_t) / (delta_x ** 2)

    for k in range(0, iteracoes-1, 1):
        for i in range(1, linhas-1, delta_x):
            for j in range(1, colunas-1, delta_x):
                u[k + 1, i, j] = gamma * (u[k][i+1][j] + u[k][i-1][j] + u[k][i][j+1] + u[k][i][j-1] - 4*(u[k][i][j])) + u[k][i][j]

    return u

#2º passo: Calcular o tempo de execução

In [ ]:
import time
import statistics

In [ ]:
iteracoes = 602
linhas = 50
colunas = 50

In [ ]:
def generate_matrix_seq(iteracoes, linhas, colunas, coeficiente):
  u = init(iteracoes, linhas, colunas)
  times = []

  for i in range(1, 11):
      initial_time = time.time()
      u = calcular(u, coeficiente)
      end_time = time.time()

      total_time = end_time - initial_time
      times.append(total_time)

      print(f"Execution: {i}, Total time: {total_time:.4f} seconds")

  print(f"\nAverage execution time: {statistics.mean(times):.4f} seconds")

  return u

In [ ]:
u1 = generate_matrix_seq(iteracoes, linhas, colunas, diamante)

Execution: 1, Total time: 4.5515 seconds
Execution: 2, Total time: 4.6939 seconds
Execution: 3, Total time: 5.6963 seconds
Execution: 4, Total time: 4.4680 seconds
Execution: 5, Total time: 5.6530 seconds
Execution: 6, Total time: 4.8363 seconds
Execution: 7, Total time: 4.4819 seconds
Execution: 8, Total time: 5.7124 seconds
Execution: 9, Total time: 4.4462 seconds
Execution: 10, Total time: 5.6791 seconds

Average execution time: 5.0219 seconds


#3º passo: Aplicar a divisão da tarefa em ```calcular``` para n threads e implementar a barreira

In [ ]:
import threading

In [ ]:
k = 0 # variável global para indicar a iteração atual

In [ ]:
# função realizada por cada thread
def medir(u, linhas, colunas, barreira, id, gamma): # id é o identificador da thread
  global k

  # faço essa subtração pois calculamos a matriz desconsiderando as primeiras e últimas linhas e colunas, pois consideramos incidência de calor direta nelas
  linhas-=2
  colunas-=2

  while k<iteracoes-1:

    for cont in range(id, linhas*colunas, num_threads):
      i = cont // colunas
      j = cont % colunas

      i+=1
      j+=1

      u[k + 1, i, j] = gamma * (u[k][i+1][j] + u[k][i-1][j] + u[k][i][j+1] + u[k][i][j-1] - 4*(u[k][i][j])) + u[k][i][j]

    barreira.wait() # espero todas as threads fazerem os cálculos

In [ ]:
def aumenta_k():
  global k
  k+=1

In [ ]:
def calcular_conc(u, coeficiente, num_threads):
  global k

  iteracoes, linhas, colunas = u.shape

  alpha = coeficiente
  delta_x = 1

  delta_t = ((delta_x-0.05) ** 2)/(4 * (alpha+0.05)) # pequeno ruído no alfa e no delta x para evitar que eles sejam cancelados na hora de calcular o gamma
  gamma = (alpha * delta_t) / (delta_x ** 2)

  threads = []
  barreira = threading.Barrier(num_threads, action=aumenta_k)
  k = 0

  for i in range(num_threads):
    thread = threading.Thread(target=medir, args=(u, linhas, colunas, barreira, i, gamma))
    threads.append(thread)
    thread.start()

  for thread in threads:
    thread.join()

  return u

In [ ]:
def generate_matrix_conc(iteracoes, linhas, colunas, coeficiente, num_threads):
  u = init(iteracoes, linhas, colunas)
  times = []

  for i in range(1, 11):
    initial_time = time.time()
    u = calcular_conc(u, coeficiente, num_threads)
    end_time = time.time()

    total_time = end_time - initial_time
    times.append(total_time)

    print(f"Execution: {i}, Total time: {total_time:.4f} seconds")

  print(f"\nAverage execution time: {statistics.mean(times):.4f} seconds")

  return u

#4º passo: Teste de desempenho da solução concorrente com a mesma dimensão da solução sequencial e calcular norma de Frobenius

Iterações = 600

Linhas = 100

Colunas = 100

In [ ]:
iteracoes = 600
linhas = 100
colunas = 100
all_threads = [1, 2, 4, 8]

print("SEQUENCIAL\n")
u1 = generate_matrix_seq(iteracoes, linhas, colunas, diamante)

for num_threads in all_threads:
    print(f"\nCONCORRENTE ({num_threads} threads)\n")
    u2 = generate_matrix_conc(iteracoes, linhas, colunas, diamante, num_threads)


SEQUENCIAL

Execution: 1, Total time: 22.6235 seconds
Execution: 2, Total time: 22.3417 seconds
Execution: 3, Total time: 20.0272 seconds
Execution: 4, Total time: 21.2566 seconds
Execution: 5, Total time: 21.9440 seconds
Execution: 6, Total time: 21.5813 seconds
Execution: 7, Total time: 21.3600 seconds
Execution: 8, Total time: 20.9117 seconds
Execution: 9, Total time: 19.9656 seconds
Execution: 10, Total time: 21.0352 seconds

Average execution time: 21.3047 seconds

CONCORRENTE (1 threads)

Execution: 1, Total time: 22.9358 seconds
Execution: 2, Total time: 22.7388 seconds
Execution: 3, Total time: 21.6163 seconds
Execution: 4, Total time: 22.7848 seconds
Execution: 5, Total time: 22.9458 seconds
Execution: 6, Total time: 22.9212 seconds
Execution: 7, Total time: 22.9034 seconds
Execution: 8, Total time: 21.6387 seconds
Execution: 9, Total time: 23.0769 seconds
Execution: 10, Total time: 22.9557 seconds

Average execution time: 22.6517 seconds

CONCORRENTE (2 threads)

Execution: 1

In [ ]:
u1 = np.asarray(u1[iteracoes-1])
u2 = np.asarray(u2[iteracoes-1])
matrix_dif = u1 - u2

print(np.linalg.norm(matrix_dif, ord='fro')) # norma de Frobenius de: (matriz concorrente de 8 threads - matriz sequencial)

0.0


Iterações = 1000

Linhas = 100

Colunas = 100

In [ ]:
iteracoes = 1000
linhas = 100
colunas = 100
all_threads = [1, 2, 4, 8]

print("SEQUENCIAL\n")
u1 = generate_matrix_seq(iteracoes, linhas, colunas, diamante)

for num_threads in all_threads:
    print(f"\nCONCORRENTE ({num_threads} threads)\n")
    u2 = generate_matrix_conc(iteracoes, linhas, colunas, diamante, num_threads)


SEQUENCIAL

Execution: 1, Total time: 34.1415 seconds
Execution: 2, Total time: 33.6960 seconds
Execution: 3, Total time: 34.5349 seconds
Execution: 4, Total time: 34.5490 seconds
Execution: 5, Total time: 33.5109 seconds
Execution: 6, Total time: 34.7699 seconds
Execution: 7, Total time: 34.6663 seconds
Execution: 8, Total time: 34.8723 seconds
Execution: 9, Total time: 33.5987 seconds
Execution: 10, Total time: 34.8306 seconds

Average execution time: 34.3170 seconds

CONCORRENTE (1 threads)

Execution: 1, Total time: 37.4190 seconds
Execution: 2, Total time: 37.4164 seconds
Execution: 3, Total time: 37.8513 seconds
Execution: 4, Total time: 37.9432 seconds
Execution: 5, Total time: 37.6703 seconds
Execution: 6, Total time: 37.7909 seconds
Execution: 7, Total time: 37.6391 seconds
Execution: 8, Total time: 37.7587 seconds
Execution: 9, Total time: 37.7016 seconds
Execution: 10, Total time: 37.6120 seconds

Average execution time: 37.6803 seconds

CONCORRENTE (2 threads)

Execution: 1

In [ ]:
u1 = np.asarray(u1[iteracoes-1])
u2 = np.asarray(u2[iteracoes-1])
matrix_dif = u1 - u2

print(np.linalg.norm(matrix_dif, ord='fro')) # norma de Frobenius de: (matriz concorrente de 8 threads - matriz sequencial)

0.0


Iterações = 2000

Linhas = 200

Colunas = 200

In [ ]:
iteracoes = 2000
linhas = 200
colunas = 200
all_threads = [1, 2, 4, 8]

print("SEQUENCIAL\n")
u1 = generate_matrix_seq(iteracoes, linhas, colunas, diamante) # essa será a matriz que será escrita no arquivo 'python_sequencial.txt'

for num_threads in all_threads:
    print(f"\nCONCORRENTE ({num_threads} threads)\n")
    u2 = generate_matrix_conc(iteracoes, linhas, colunas, diamante, num_threads) # a matriz que será escrita no arquivo 'python_concorrente.txt' é aquela computada usando 8 threads


SEQUENCIAL

Execution: 1, Total time: 279.2122 seconds
Execution: 2, Total time: 280.0156 seconds
Execution: 3, Total time: 278.8684 seconds
Execution: 4, Total time: 278.8308 seconds
Execution: 5, Total time: 278.6448 seconds
Execution: 6, Total time: 279.9469 seconds
Execution: 7, Total time: 279.2529 seconds
Execution: 8, Total time: 278.9805 seconds
Execution: 9, Total time: 278.5006 seconds
Execution: 10, Total time: 278.8510 seconds

Average execution time: 279.1104 seconds

CONCORRENTE (1 threads)

Execution: 1, Total time: 305.7218 seconds
Execution: 2, Total time: 307.2225 seconds
Execution: 3, Total time: 307.8320 seconds
Execution: 4, Total time: 307.8459 seconds
Execution: 5, Total time: 308.6108 seconds
Execution: 6, Total time: 307.8079 seconds
Execution: 7, Total time: 311.2968 seconds
Execution: 8, Total time: 309.5804 seconds
Execution: 9, Total time: 307.8740 seconds
Execution: 10, Total time: 309.3174 seconds

Average execution time: 308.3110 seconds

CONCORRENTE (2 

In [ ]:
u1 = np.asarray(u1[iteracoes-1])
u2 = np.asarray(u2[iteracoes-1])
matrix_dif = u1 - u2

print(np.linalg.norm(matrix_dif, ord='fro')) # norma de Frobenius de: (matriz concorrente de 8 threads - matriz sequencial)

0.0


#5º passo: Escrever as matrizes em arquivos .txt

In [50]:
#escrever em arquivo

f_seq = open("python_sequencial.txt", "w") # usar w para write normal ou wb para binario
f_seq.write(str(iteracoes))
f_seq.write(" ")
f_seq.write(str(linhas))
f_seq.write(" ")
f_seq.write(str(colunas))
f_seq.write("\n")
for i in range(linhas):
    for j in range(colunas):
        f_seq.write(f"{u1[i][j]:.6f} ")

f_seq.close()


f_conc = open("python_concorrente.txt", "w") # usar w para write normal ou wb para binario
f_conc.write(str(iteracoes))
f_conc.write(" ")
f_conc.write(str(linhas))
f_conc.write(" ")
f_conc.write(str(colunas))
f_conc.write("\n")
for i in range(linhas):
    for j in range(colunas):
        f_conc.write(f"{u2[i][j]:.6f} ")

f_conc.close()